# Cleaning Cryptocurrency data

In this notebook, we are going to merge the data as best as possible with the previous CSV files opened in the previous notebook. As we know, all of them except `bitcoin_dataset.csv` and `ethereum_dataset.csv` have the same columns. These columns are the following:

* Date: date of observation
* Open: Opening price on the given day
* High: Highest price on the given day
* Low: Lowest price on the given day
* Close: Closing price on the given day
* Volume: Volume of transactions on the given day
* Market Cap: Market capitalization in USD

In [1]:
from pathlib import Path

import pandas as pd

In [2]:
DATA_PATH = Path('../../data/raw/cryptocurrencypricehistory')
PROCESSED_DATA_PATH = Path('../../data/processed/cryptocurrencypricehistory')

PROCESSED_DATA_PATH.mkdir(exist_ok=True, parents=True)

Then we can see the different files that need to be cleaned by us.

In [8]:
[f'{o.stem}{o.suffix}' for o in  DATA_PATH.iterdir()]

['bitcoin_cash_price.csv',
 'bitcoin_dataset.csv',
 'bitcoin_price.csv',
 'bitconnect_price.csv',
 'dash_price.csv',
 'ethereum_classic_price.csv',
 'ethereum_dataset.csv',
 'ethereum_price.csv',
 'iota_price.csv',
 'litecoin_price.csv',
 'monero_price.csv',
 'nem_price.csv',
 'neo_price.csv',
 'numeraire_price.csv',
 'omisego_price.csv',
 'qtum_price.csv',
 'ripple_price.csv',
 'stratis_price.csv',
 'waves_price.csv']

For instance, if we open a file and read then, we can see the different attributes, as mentioned before. 

In [4]:
df_0 = pd.read_csv(DATA_PATH / 'bitcoin_cash_price.csv', index_col='Date')

df_0.head()

,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
"Sep 05, 2017",514.90,550.95,458.78,541.71,"338,978,000","8,527,100,000"
"Sep 04, 2017",608.26,608.26,500.75,517.24,"328,957,000","10,072,200,000"
"Sep 03, 2017",578.27,617.41,563.59,607.43,"344,862,000","9,574,520,000"
"Sep 02, 2017",621.96,642.05,560.58,575.90,"350,478,000","10,297,000,000"
"Sep 01, 2017",588.40,645.52,586.73,622.17,"393,839,000","9,740,460,000"


The value that we take into account on this table will be the `Close` value.

Now, we are going to use the `merge` method to concatenate each column of the dataset by `date`. For each currency, we will get the `Close` value. Let's try with the previous example and all of the currencies with the same columns.

After that, for each column, we will rename the column to each cryptocurrency name to distinguish them.

In [16]:
col_list = ["Date", "Close"]
num = 1

path = r'../../data/raw/cryptocurrencypricehistory'
files = DATA_PATH.glob('*.csv')

df_cc = pd.read_csv(DATA_PATH / 'bitcoin_cash_price.csv', index_col='Date', usecols=col_list)
df_cc.rename(columns={df_cc.columns[0]:'bitcoin_cash_price'}, inplace=True)

for filename in files:
    if filename.stem != 'bitcoin_dataset' and filename.stem != 'ethereum_dataset':
        nf = filename.stem
        df = pd.read_csv(filename, index_col='Date', usecols=col_list)
        #df_cc = pd.concat([df_cc, df], axis=1)
        df_cc = df_cc.merge(df, 'left', 'Date')

        df_cc.rename(columns={df_cc.columns[num]:nf}, inplace=True)

        num += 1

df_cc.head()

,bitcoin_cash_price,bitcoin_cash_price,bitcoin_price,bitconnect_price,dash_price,ethereum_classic_price,ethereum_price,iota_price,litecoin_price,monero_price,nem_price,neo_price,numeraire_price,omisego_price,qtum_price,ripple_price,stratis_price,waves_price
Date,,,,,,,,,,,,,,,,,,
"Sep 05, 2017",541.71,541.71,4376.53,129.42,327.23,16.58,312.99,0.613085,71.29,118.82,0.286227,22.80,21.53,10.98,11.71,0.215189,6.03,4.98
"Sep 04, 2017",517.24,517.24,4236.31,114.13,316.13,15.84,295.17,0.566472,65.21,106.17,0.273220,21.83,20.74,8.78,10.98,0.204968,5.77,4.66
"Sep 03, 2017",607.43,607.43,4582.96,130.99,356.39,18.57,347.48,0.743968,76.84,126.01,0.307264,30.32,26.90,10.57,15.29,0.228811,6.59,5.20
"Sep 02, 2017",575.90,575.90,4578.77,131.33,350.17,20.08,348.98,0.695547,79.02,124.80,0.295884,31.72,27.24,10.80,16.39,0.226669,6.34,5.23
"Sep 01, 2017",622.17,622.17,4892.01,140.97,393.35,21.94,387.74,0.807778,86.04,141.20,0.332310,32.01,32.45,11.97,18.26,0.248479,7.25,5.74


Now, we have a table of all close prices of each type of cryptocurrency except the `bitcoin_dataset.csv` and `ethereum_dataset.csv`. 

Then we are going to check if exists NaN values into the rows.

In [17]:
print('Rows containing NaN:', df_cc.isna().any(axis=1).sum())
df_cc.isna().any(axis=0)

Rows containing NaN: 15


bitcoin_cash_price        False
bitcoin_cash_price        False
bitcoin_price             False
bitconnect_price          False
dash_price                False
ethereum_classic_price    False
ethereum_price            False
iota_price                False
litecoin_price            False
monero_price              False
nem_price                 False
neo_price                 False
numeraire_price            True
omisego_price              True
qtum_price                 True
ripple_price              False
stratis_price             False
waves_price               False
dtype: bool

Also, we are going to repeat the process for `bitcoin_dataset.csv` and `ethereum_dataset.csv`. 

In [18]:
df_bit = pd.read_csv(DATA_PATH / 'bitcoin_dataset.csv', index_col='Date' )
print('--- Bitcoin Dataset ---')
print('Rows containing NaN:', df_bit.isna().any(axis=1).sum())
df_bit.isna().any(axis=0)

--- Bitcoin Dataset ---
Rows containing NaN: 478


btc_market_price                                       False
btc_total_bitcoins                                     False
btc_market_cap                                         False
btc_trade_volume                                        True
btc_blocks_size                                        False
btc_avg_block_size                                     False
btc_n_orphaned_blocks                                  False
btc_n_transactions_per_block                           False
btc_median_confirmation_time                           False
btc_hash_rate                                          False
btc_difficulty                                         False
btc_miners_revenue                                     False
btc_transaction_fees                                   False
btc_cost_per_transaction_percent                       False
btc_cost_per_transaction                               False
btc_n_unique_addresses                                 False
btc_n_transactions      

In [19]:
df_eth = pd.read_csv(DATA_PATH / 'ethereum_dataset.csv', index_col='Date(UTC)')
print('--- Ethereum Dataset ---')
print('Rows containing NaN:', df_eth.isna().any(axis=1).sum())
df_eth.isna().any(axis=0)

--- Ethereum Dataset ---
Rows containing NaN: 769


UnixTimeStamp        False
eth_etherprice       False
eth_tx               False
eth_address          False
eth_supply           False
eth_marketcap        False
eth_hashrate         False
eth_difficulty       False
eth_blocks           False
eth_uncles           False
eth_blocksize        False
eth_blocktime        False
eth_gasprice         False
eth_gaslimit         False
eth_gasused          False
eth_ethersupply      False
eth_chaindatasize     True
eth_ens_register      True
dtype: bool

As we can see, some of them have Nan values that they will be deleted in order to clean the data as best as possible. As we know in this case, there are two options to clean the NaN values:

* Fill each gap with a determined number
* Remove the row of this day

Delete a row is not a solution for the close values, we would rather not remove these rows because other currencies can have value for this day, so we decided to fill this value changing the value to 0.

In [20]:
df_cc.fillna(0., inplace=True)

On the `bitcoin_dataset.csv` and `ethereum_dataset.csv` we can also replace these rows that contain NaN values. As we've seen before, the first one have around 500 NaN values, all of them into one column, if we remove them we will not have many values to extract results in the future. The second one, has around 800 NaN values, and also we will not remove these rows because we prefer to keep data. 

In [21]:
df_bit.fillna(0., inplace=True)
df_eth.fillna(0., inplace=True)

Finally, we only save the data into a new csv file that we will save into the processed data directory defined in the previous steps.

In [22]:
df_cc.to_csv(PROCESSED_DATA_PATH / f'cryptocurrency_close_values.csv')
df_bit.to_csv(PROCESSED_DATA_PATH / f'bitcoin_dataset.csv')
df_eth.to_csv(PROCESSED_DATA_PATH / f'ethereum_dataset.csv')

df_cc.index.names = ['Date']
df_cc.head()

,bitcoin_cash_price,bitcoin_cash_price,bitcoin_price,bitconnect_price,dash_price,ethereum_classic_price,ethereum_price,iota_price,litecoin_price,monero_price,nem_price,neo_price,numeraire_price,omisego_price,qtum_price,ripple_price,stratis_price,waves_price
Date,,,,,,,,,,,,,,,,,,
"Sep 05, 2017",541.71,541.71,4376.53,129.42,327.23,16.58,312.99,0.613085,71.29,118.82,0.286227,22.80,21.53,10.98,11.71,0.215189,6.03,4.98
"Sep 04, 2017",517.24,517.24,4236.31,114.13,316.13,15.84,295.17,0.566472,65.21,106.17,0.273220,21.83,20.74,8.78,10.98,0.204968,5.77,4.66
"Sep 03, 2017",607.43,607.43,4582.96,130.99,356.39,18.57,347.48,0.743968,76.84,126.01,0.307264,30.32,26.90,10.57,15.29,0.228811,6.59,5.20
"Sep 02, 2017",575.90,575.90,4578.77,131.33,350.17,20.08,348.98,0.695547,79.02,124.80,0.295884,31.72,27.24,10.80,16.39,0.226669,6.34,5.23
"Sep 01, 2017",622.17,622.17,4892.01,140.97,393.35,21.94,387.74,0.807778,86.04,141.20,0.332310,32.01,32.45,11.97,18.26,0.248479,7.25,5.74


## References

\[1\] [Matplotlib documentation](https://matplotlib.org/)

\[2\] [Scikit Learn Documentation](https://scikit-learn.org/stable/)